In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [2]:
def extract_features(file_path, mfcc=True, chroma=True, mel=True):
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    features = []
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13), axis=1)
        features.extend(mfccs)
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate), axis=1)
        features.extend(chroma)
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate), axis=1)
        features.extend(mel)
    return features

In [3]:
def preprocess_dataset(data_path, labels):
    X = []
    y = []
    for label, folder in enumerate(labels):
        for filename in os.listdir(os.path.join(data_path, folder)):
            file_path = os.path.join(data_path, folder, filename)
            features = extract_features(file_path)
            X.append(features)
            y.append(label)
    return np.array(X), np.array(y)

In [4]:
data_path = "for-rerecorded/training"
labels = ["fake", "real"]


In [5]:
X, y = preprocess_dataset(data_path, labels)

In [6]:
x_train=X

In [7]:
y_train=y

In [9]:
data_path1 = "for-rerecorded/validation"




In [10]:
X1, y1 = preprocess_dataset(data_path1, labels)

In [11]:
x_val=X1

In [12]:
y_val=y1

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(x_train.shape[1], 1)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
X_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
X_val = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))


In [18]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
319/319 [==============================] - 14s 22ms/step - loss: 0.6808 - accuracy: 0.5624 - val_loss: 0.7061 - val_accuracy: 0.5361
Epoch 2/10
319/319 [==============================] - 6s 19ms/step - loss: 0.6427 - accuracy: 0.6408 - val_loss: 0.6329 - val_accuracy: 0.6836
Epoch 3/10
319/319 [==============================] - 6s 20ms/step - loss: 0.5214 - accuracy: 0.7505 - val_loss: 0.4437 - val_accuracy: 0.7972
Epoch 4/10
319/319 [==============================] - 6s 19ms/step - loss: 0.4399 - accuracy: 0.8028 - val_loss: 0.4716 - val_accuracy: 0.7865
Epoch 5/10
319/319 [==============================] - 6s 19ms/step - loss: 0.4062 - accuracy: 0.8151 - val_loss: 0.4255 - val_accuracy: 0.8186
Epoch 6/10
319/319 [==============================] - 6s 19ms/step - loss: 0.3760 - accuracy: 0.8315 - val_loss: 0.4123 - val_accuracy: 0.8195
Epoch 7/10
319/319 [==============================] - 6s 19ms/step - loss: 0.3492 - accuracy: 0.8438 - val_loss: 0.3420 - val_accuracy: 0.848

In [19]:
model.fit(X_train, y_train, epochs=15, batch_size=16, validation_data=(X_val, y_val))

Epoch 1/15
638/638 [==============================] - 11s 17ms/step - loss: 0.3073 - accuracy: 0.8710 - val_loss: 0.2543 - val_accuracy: 0.8944
Epoch 2/15
638/638 [==============================] - 10s 16ms/step - loss: 0.2663 - accuracy: 0.8927 - val_loss: 0.2310 - val_accuracy: 0.9113
Epoch 3/15
638/638 [==============================] - 10s 16ms/step - loss: 0.2725 - accuracy: 0.8888 - val_loss: 0.3640 - val_accuracy: 0.8516
Epoch 4/15
638/638 [==============================] - 10s 15ms/step - loss: 0.2490 - accuracy: 0.9031 - val_loss: 0.2421 - val_accuracy: 0.9046
Epoch 5/15
638/638 [==============================] - 10s 15ms/step - loss: 0.2162 - accuracy: 0.9175 - val_loss: 0.2343 - val_accuracy: 0.9046
Epoch 6/15
638/638 [==============================] - 10s 15ms/step - loss: 0.1888 - accuracy: 0.9300 - val_loss: 0.2478 - val_accuracy: 0.9091
Epoch 7/15
638/638 [==============================] - 10s 15ms/step - loss: 0.1770 - accuracy: 0.9358 - val_loss: 0.2094 - val_accuracy:

In [177]:
new_audio_file_path = "for-rerecorded/testing/fake/recording13022.wav_norm_mono.wav"

In [178]:
new_audio_features = extract_features(new_audio_file_path)

In [179]:
new_audio_features=np.array(new_audio_features)

In [180]:
new_audio_features = np.reshape(new_audio_features, (1, new_audio_features.shape[0], 1))

In [181]:
predictions = model.predict(new_audio_features)

1/1 [==============================] - 0s 24ms/step


In [182]:
predicted_class = 1 if predictions[0][0] > 0.5 else 0

In [183]:
print("Predicted Class:", labels[predicted_class])

Predicted Class: fake


In [35]:
model.save('deepfakeaudio1.h5')

In [36]:
model1 = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(x_train.shape[1], 1)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

In [37]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [38]:
model1.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
319/319 [==============================] - 8s 22ms/step - loss: 0.6588 - accuracy: 0.6114 - val_loss: 0.6382 - val_accuracy: 0.6502
Epoch 2/10
319/319 [==============================] - 5s 15ms/step - loss: 0.6403 - accuracy: 0.6457 - val_loss: 0.6819 - val_accuracy: 0.5735
Epoch 3/10
319/319 [==============================] - 5s 14ms/step - loss: 0.6285 - accuracy: 0.6611 - val_loss: 0.6317 - val_accuracy: 0.6649
Epoch 4/10
319/319 [==============================] - 5s 14ms/step - loss: 0.5446 - accuracy: 0.7317 - val_loss: 0.4841 - val_accuracy: 0.7857
Epoch 5/10
319/319 [==============================] - 5s 14ms/step - loss: 0.4553 - accuracy: 0.7884 - val_loss: 0.4259 - val_accuracy: 0.8066
Epoch 6/10
319/319 [==============================] - 5s 14ms/step - loss: 0.4048 - accuracy: 0.8135 - val_loss: 0.4145 - val_accuracy: 0.8070
Epoch 7/10
319/319 [==============================] - 4s 14ms/step - loss: 0.3812 - accuracy: 0.8212 - val_loss: 0.3598 - val_accuracy: 0.8351

In [39]:
model1.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/10
160/160 [==============================] - 4s 19ms/step - loss: 0.2966 - accuracy: 0.8770 - val_loss: 0.2975 - val_accuracy: 0.8877
Epoch 2/10
160/160 [==============================] - 3s 17ms/step - loss: 0.2709 - accuracy: 0.8905 - val_loss: 0.2847 - val_accuracy: 0.8895
Epoch 3/10
160/160 [==============================] - 3s 17ms/step - loss: 0.2720 - accuracy: 0.8925 - val_loss: 0.3404 - val_accuracy: 0.8636
Epoch 4/10
160/160 [==============================] - 3s 17ms/step - loss: 0.2429 - accuracy: 0.9056 - val_loss: 0.2670 - val_accuracy: 0.8980
Epoch 5/10
160/160 [==============================] - 3s 16ms/step - loss: 0.2609 - accuracy: 0.8942 - val_loss: 0.2678 - val_accuracy: 0.8922
Epoch 6/10
160/160 [==============================] - 3s 17ms/step - loss: 0.2266 - accuracy: 0.9111 - val_loss: 0.2720 - val_accuracy: 0.8975
Epoch 7/10
160/160 [==============================] - 3s 16ms/step - loss: 0.2263 - accuracy: 0.9125 - val_loss: 0.2746 - val_accuracy: 0.8988

In [41]:
model3 = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(x_train.shape[1], 1)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

In [42]:
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [43]:
model3.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val),steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 5s 34ms/step - loss: 0.6845 - accuracy: 0.5628 - val_loss: 0.6606 - val_accuracy: 0.6355
Epoch 2/10
100/100 [==============================] - 2s 23ms/step - loss: 0.6645 - accuracy: 0.6034 - val_loss: 0.6883 - val_accuracy: 0.5169
Epoch 3/10
100/100 [==============================] - 2s 21ms/step - loss: 0.6373 - accuracy: 0.6422 - val_loss: 0.6509 - val_accuracy: 0.6350
Epoch 4/10
100/100 [==============================] - 2s 20ms/step - loss: 0.6445 - accuracy: 0.6397 - val_loss: 0.6486 - val_accuracy: 0.6279
Epoch 5/10
100/100 [==============================] - 2s 20ms/step - loss: 0.6380 - accuracy: 0.6553 - val_loss: 0.6432 - val_accuracy: 0.6426
Epoch 6/10
100/100 [==============================] - 2s 19ms/step - loss: 0.5969 - accuracy: 0.6997 - val_loss: 0.5766 - val_accuracy: 0.7130
Epoch 7/10
100/100 [==============================] - 2s 20ms/step - loss: 0.5983 - accuracy: 0.6978 - val_loss: 0.5687 - val_accuracy: 0.7201

In [44]:
model3.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val),steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 3s 31ms/step - loss: 0.4101 - accuracy: 0.7991 - val_loss: 0.4111 - val_accuracy: 0.8177
Epoch 2/10
100/100 [==============================] - 2s 24ms/step - loss: 0.4218 - accuracy: 0.8078 - val_loss: 0.4041 - val_accuracy: 0.8012
Epoch 3/10
100/100 [==============================] - 2s 20ms/step - loss: 0.4214 - accuracy: 0.8069 - val_loss: 0.3990 - val_accuracy: 0.8137
Epoch 4/10
100/100 [==============================] - 2s 18ms/step - loss: 0.3833 - accuracy: 0.8244 - val_loss: 0.3791 - val_accuracy: 0.8271
Epoch 5/10
100/100 [==============================] - 2s 17ms/step - loss: 0.3796 - accuracy: 0.8297 - val_loss: 0.4308 - val_accuracy: 0.7977
Epoch 6/10
100/100 [==============================] - 2s 18ms/step - loss: 0.3940 - accuracy: 0.8203 - val_loss: 0.3635 - val_accuracy: 0.8422
Epoch 7/10
100/100 [==============================] - 2s 18ms/step - loss: 0.3738 - accuracy: 0.8338 - val_loss: 0.3549 - val_accuracy: 0.8414

In [45]:
model3.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val),steps_per_epoch=100)

Epoch 1/20
100/100 [==============================] - 2s 21ms/step - loss: 0.3489 - accuracy: 0.8528 - val_loss: 0.3320 - val_accuracy: 0.8587
Epoch 2/20
100/100 [==============================] - 2s 19ms/step - loss: 0.3058 - accuracy: 0.8797 - val_loss: 0.3734 - val_accuracy: 0.8503
Epoch 3/20
100/100 [==============================] - 2s 18ms/step - loss: 0.3232 - accuracy: 0.8681 - val_loss: 0.3479 - val_accuracy: 0.8458
Epoch 4/20
100/100 [==============================] - 2s 18ms/step - loss: 0.3178 - accuracy: 0.8734 - val_loss: 0.3036 - val_accuracy: 0.8832
Epoch 5/20
100/100 [==============================] - 2s 18ms/step - loss: 0.2995 - accuracy: 0.8809 - val_loss: 0.2921 - val_accuracy: 0.8819
Epoch 6/20
100/100 [==============================] - 2s 18ms/step - loss: 0.2889 - accuracy: 0.8819 - val_loss: 0.2852 - val_accuracy: 0.8837
Epoch 7/20
100/100 [==============================] - 2s 18ms/step - loss: 0.2789 - accuracy: 0.8872 - val_loss: 0.2753 - val_accuracy: 0.8855

In [46]:
model3.save('deepfakeaudio2.h5')

In [58]:
model4 = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(x_train.shape[1], 1)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

In [59]:
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [60]:
from tensorflow.keras.callbacks import EarlyStopping

In [61]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [62]:
model4.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/50
319/319 [==============================] - 8s 22ms/step - loss: 0.6607 - accuracy: 0.6145 - val_loss: 0.6329 - val_accuracy: 0.6640
Epoch 2/50
319/319 [==============================] - 5s 15ms/step - loss: 0.6325 - accuracy: 0.6603 - val_loss: 0.6315 - val_accuracy: 0.6640
Epoch 3/50
319/319 [==============================] - 5s 15ms/step - loss: 0.5818 - accuracy: 0.7096 - val_loss: 0.5319 - val_accuracy: 0.7527
Epoch 4/50
319/319 [==============================] - 5s 14ms/step - loss: 0.4876 - accuracy: 0.7745 - val_loss: 0.4339 - val_accuracy: 0.8084
Epoch 5/50
319/319 [==============================] - 5s 14ms/step - loss: 0.4353 - accuracy: 0.7980 - val_loss: 0.4213 - val_accuracy: 0.8066
Epoch 6/50
319/319 [==============================] - 5s 15ms/step - loss: 0.4133 - accuracy: 0.8091 - val_loss: 0.4957 - val_accuracy: 0.7750
Epoch 7/50
319/319 [==============================] - 5s 15ms/step - loss: 0.3749 - accuracy: 0.8336 - val_loss: 0.3795 - val_accuracy: 0.8200

In [70]:
model4 = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, input_shape=(x_train.shape[1], 1)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

In [71]:
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [72]:
model4.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/50
319/319 [==============================] - 8s 21ms/step - loss: 0.6613 - accuracy: 0.6029 - val_loss: 0.6376 - val_accuracy: 0.6511
Epoch 2/50
319/319 [==============================] - 5s 16ms/step - loss: 0.6376 - accuracy: 0.6535 - val_loss: 0.6313 - val_accuracy: 0.6586
Epoch 3/50
319/319 [==============================] - 5s 15ms/step - loss: 0.5963 - accuracy: 0.6964 - val_loss: 0.5489 - val_accuracy: 0.7259
Epoch 4/50
319/319 [==============================] - 5s 14ms/step - loss: 0.5140 - accuracy: 0.7429 - val_loss: 0.4708 - val_accuracy: 0.7683
Epoch 5/50
319/319 [==============================] - 5s 14ms/step - loss: 0.4776 - accuracy: 0.7731 - val_loss: 0.4588 - val_accuracy: 0.7714
Epoch 6/50
319/319 [==============================] - 5s 15ms/step - loss: 0.4401 - accuracy: 0.7911 - val_loss: 0.4650 - val_accuracy: 0.7758
Epoch 7/50
319/319 [==============================] - 5s 15ms/step - loss: 0.4390 - accuracy: 0.7882 - val_loss: 0.4183 - val_accuracy: 0.7999

In [73]:
model4 = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, input_shape=(x_train.shape[1], 1)),
    
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

In [74]:
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [75]:
model4.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/50
319/319 [==============================] - 8s 21ms/step - loss: 0.6569 - accuracy: 0.6125 - val_loss: 0.6540 - val_accuracy: 0.6243
Epoch 2/50
319/319 [==============================] - 5s 15ms/step - loss: 0.6241 - accuracy: 0.6659 - val_loss: 0.5969 - val_accuracy: 0.6988
Epoch 3/50
319/319 [==============================] - 5s 15ms/step - loss: 0.5588 - accuracy: 0.7303 - val_loss: 0.5530 - val_accuracy: 0.7166
Epoch 4/50
319/319 [==============================] - 5s 15ms/step - loss: 0.5174 - accuracy: 0.7460 - val_loss: 0.5233 - val_accuracy: 0.7420
Epoch 5/50
319/319 [==============================] - 5s 15ms/step - loss: 0.4502 - accuracy: 0.7872 - val_loss: 0.4807 - val_accuracy: 0.7709
Epoch 6/50
319/319 [==============================] - 5s 14ms/step - loss: 0.4150 - accuracy: 0.8060 - val_loss: 0.4240 - val_accuracy: 0.7946
Epoch 7/50
319/319 [==============================] - 5s 15ms/step - loss: 0.4040 - accuracy: 0.8083 - val_loss: 0.3823 - val_accuracy: 0.8146

In [76]:
model4.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping],steps_per_epoch=100)

Epoch 1/100
100/100 [==============================] - 3s 28ms/step - loss: 0.2917 - accuracy: 0.8853 - val_loss: 0.3161 - val_accuracy: 0.8748
Epoch 2/100
100/100 [==============================] - 2s 22ms/step - loss: 0.2856 - accuracy: 0.8872 - val_loss: 0.2749 - val_accuracy: 0.8930
Epoch 3/100
100/100 [==============================] - 2s 21ms/step - loss: 0.2766 - accuracy: 0.8853 - val_loss: 0.2785 - val_accuracy: 0.8899
Epoch 4/100
100/100 [==============================] - 2s 19ms/step - loss: 0.2909 - accuracy: 0.8813 - val_loss: 0.2882 - val_accuracy: 0.8873
Epoch 5/100
100/100 [==============================] - 2s 19ms/step - loss: 0.2751 - accuracy: 0.8925 - val_loss: 0.2867 - val_accuracy: 0.8935


In [77]:
model4.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping],steps_per_epoch=100)

Epoch 1/100
100/100 [==============================] - 3s 28ms/step - loss: 0.2656 - accuracy: 0.8922 - val_loss: 0.2797 - val_accuracy: 0.8890
Epoch 2/100
100/100 [==============================] - 2s 23ms/step - loss: 0.2784 - accuracy: 0.8894 - val_loss: 0.2701 - val_accuracy: 0.8971
Epoch 3/100
100/100 [==============================] - 2s 20ms/step - loss: 0.2902 - accuracy: 0.8825 - val_loss: 0.2655 - val_accuracy: 0.8988
Epoch 4/100
100/100 [==============================] - 2s 19ms/step - loss: 0.2686 - accuracy: 0.8925 - val_loss: 0.2748 - val_accuracy: 0.8957
Epoch 5/100
100/100 [==============================] - 2s 19ms/step - loss: 0.2598 - accuracy: 0.8947 - val_loss: 0.2591 - val_accuracy: 0.8980
Epoch 6/100
100/100 [==============================] - 2s 19ms/step - loss: 0.2848 - accuracy: 0.8838 - val_loss: 0.3001 - val_accuracy: 0.8801
Epoch 7/100
100/100 [==============================] - 2s 19ms/step - loss: 0.2613 - accuracy: 0.8984 - val_loss: 0.2850 - val_accuracy:

In [79]:
model4.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 3s 27ms/step - loss: 0.2721 - accuracy: 0.8913 - val_loss: 0.2554 - val_accuracy: 0.9015
Epoch 2/10
100/100 [==============================] - 2s 24ms/step - loss: 0.2496 - accuracy: 0.9038 - val_loss: 0.3109 - val_accuracy: 0.8757
Epoch 3/10
100/100 [==============================] - 2s 21ms/step - loss: 0.2644 - accuracy: 0.8966 - val_loss: 0.2531 - val_accuracy: 0.9046
Epoch 4/10
100/100 [==============================] - 2s 19ms/step - loss: 0.2729 - accuracy: 0.8906 - val_loss: 0.2667 - val_accuracy: 0.8966
Epoch 5/10
100/100 [==============================] - 2s 18ms/step - loss: 0.2609 - accuracy: 0.9016 - val_loss: 0.2884 - val_accuracy: 0.8841
Epoch 6/10
100/100 [==============================] - 2s 18ms/step - loss: 0.2474 - accuracy: 0.9038 - val_loss: 0.2600 - val_accuracy: 0.9042
Epoch 7/10
100/100 [==============================] - 2s 18ms/step - loss: 0.2481 - accuracy: 0.9069 - val_loss: 0.2480 - val_accuracy: 0.9078

In [80]:
model4.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 3s 27ms/step - loss: 0.2369 - accuracy: 0.9119 - val_loss: 0.2660 - val_accuracy: 0.9011
Epoch 2/10
100/100 [==============================] - 2s 22ms/step - loss: 0.2298 - accuracy: 0.9087 - val_loss: 0.2482 - val_accuracy: 0.9033
Epoch 3/10
100/100 [==============================] - 2s 20ms/step - loss: 0.2568 - accuracy: 0.8991 - val_loss: 0.2492 - val_accuracy: 0.9042
Epoch 4/10
100/100 [==============================] - 2s 18ms/step - loss: 0.2419 - accuracy: 0.9038 - val_loss: 0.2547 - val_accuracy: 0.8984
Epoch 5/10
100/100 [==============================] - 2s 18ms/step - loss: 0.2434 - accuracy: 0.8997 - val_loss: 0.4370 - val_accuracy: 0.8119
Epoch 6/10
100/100 [==============================] - 2s 18ms/step - loss: 0.2454 - accuracy: 0.9025 - val_loss: 0.2563 - val_accuracy: 0.8975
Epoch 7/10
100/100 [==============================] - 2s 18ms/step - loss: 0.2367 - accuracy: 0.9075 - val_loss: 0.2670 - val_accuracy: 0.8984

In [86]:
model7 = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, input_shape=(x_train.shape[1], 1)),
    

    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),  # Add dropout with a dropout rate of 0.2

    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

In [87]:
model7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [88]:
model7.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping],steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 5s 32ms/step - loss: 0.6908 - accuracy: 0.5309 - val_loss: 0.6837 - val_accuracy: 0.6096
Epoch 2/10
100/100 [==============================] - 2s 23ms/step - loss: 0.6634 - accuracy: 0.6150 - val_loss: 0.6485 - val_accuracy: 0.6448
Epoch 3/10
100/100 [==============================] - 2s 21ms/step - loss: 0.6436 - accuracy: 0.6413 - val_loss: 0.6396 - val_accuracy: 0.6439
Epoch 4/10
100/100 [==============================] - 2s 19ms/step - loss: 0.6353 - accuracy: 0.6541 - val_loss: 0.6370 - val_accuracy: 0.6560
Epoch 5/10
100/100 [==============================] - 2s 19ms/step - loss: 0.6324 - accuracy: 0.6587 - val_loss: 0.6318 - val_accuracy: 0.6613
Epoch 6/10
100/100 [==============================] - 2s 19ms/step - loss: 0.6430 - accuracy: 0.6444 - val_loss: 0.6318 - val_accuracy: 0.6569
Epoch 7/10
100/100 [==============================] - 2s 19ms/step - loss: 0.6277 - accuracy: 0.6631 - val_loss: 0.6314 - val_accuracy: 0.6609

In [89]:
model7.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping],steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 2s 24ms/step - loss: 0.6244 - accuracy: 0.6772 - val_loss: 0.6461 - val_accuracy: 0.6377
Epoch 2/10
100/100 [==============================] - 2s 21ms/step - loss: 0.6225 - accuracy: 0.6722 - val_loss: 0.6199 - val_accuracy: 0.6760
Epoch 3/10
100/100 [==============================] - 2s 19ms/step - loss: 0.6264 - accuracy: 0.6694 - val_loss: 0.6451 - val_accuracy: 0.6221
Epoch 4/10
100/100 [==============================] - 2s 18ms/step - loss: 0.6126 - accuracy: 0.6806 - val_loss: 0.6458 - val_accuracy: 0.6283
Epoch 5/10
100/100 [==============================] - 2s 19ms/step - loss: 0.6205 - accuracy: 0.6766 - val_loss: 0.6101 - val_accuracy: 0.6760
Epoch 6/10
100/100 [==============================] - 2s 19ms/step - loss: 0.6053 - accuracy: 0.6975 - val_loss: 0.6649 - val_accuracy: 0.6119
Epoch 7/10
100/100 [==============================] - 2s 18ms/step - loss: 0.5922 - accuracy: 0.7053 - val_loss: 0.6122 - val_accuracy: 0.6881

In [90]:
model8 = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(x_train.shape[1], 1)),
    

    tf.keras.layers.Dense(32, activation='relu'),
   
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

In [91]:
model8.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [92]:
model8.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping],steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 5s 37ms/step - loss: 0.6741 - accuracy: 0.5928 - val_loss: 0.7186 - val_accuracy: 0.4483
Epoch 2/10
100/100 [==============================] - 3s 28ms/step - loss: 0.6941 - accuracy: 0.5359 - val_loss: 0.6840 - val_accuracy: 0.6007
Epoch 3/10
100/100 [==============================] - 2s 25ms/step - loss: 0.6804 - accuracy: 0.5872 - val_loss: 0.6592 - val_accuracy: 0.6435
Epoch 4/10
100/100 [==============================] - 2s 24ms/step - loss: 0.6613 - accuracy: 0.6175 - val_loss: 0.6594 - val_accuracy: 0.6038
Epoch 5/10
100/100 [==============================] - 2s 23ms/step - loss: 0.6507 - accuracy: 0.6369 - val_loss: 0.6631 - val_accuracy: 0.6096
Epoch 6/10
100/100 [==============================] - 2s 23ms/step - loss: 0.5938 - accuracy: 0.6997 - val_loss: 0.5046 - val_accuracy: 0.7616
Epoch 7/10
100/100 [==============================] - 2s 23ms/step - loss: 0.5370 - accuracy: 0.7416 - val_loss: 0.5429 - val_accuracy: 0.7246